**This file contains i) overview of project desctiption, ii) general questions and thoughts, iii) a glossary of terms, iv) some of the resources/articles we consulted in the early phase of the work (which may not all have made it into the final report as citations).**

## Overview
 - We are testing the waters; trying out methods, libraries and examples.
 - Recording our findings as we go, we aim to create an overview of some important methods in mechanistic interpretability. Hopefully, we can make some sort of lookup table or flow chart.


## End product/deliverables
3 main things (by mid/late August)
1. Presentation of our results and process - 20min
2. Write a rapport - do not underestimate the time this takes
    - What is a circuit?
    - How do we find a circuit? : First general (attr. patching), then zoom in (act. pt. and path pt.)
    - How do we verify a circuit? : Still a very new field, but causal scrubbing best so far
3. Computational essay - "a guide" = flow chart, we might (update: definitely) need separate sections
    - One file which contains out functions, preferably gathered in classes wrt the different type of models or packages.
    - A notebook with well-suited examples of the functions in action. In the same order/gen. structure as the rapport?

## Who to ask
Helene back 7th July, Ane leaves thursday afternoon and back 11th July, but be efficient


## Questions

- Jina AI (encoder-style transformers / non-causal) tilfredsstiller ikke antagelsene i TransformerLens (non-causal, post-layer norm, mm.), og det ser ikke ut som det er enkelt å lage en wrapper. Men Captum ser ut til å kunne gjøre noe (ihvertfall attribution) og circuitsvis ser veldig bra ut.

- Interpretability er (antagelig) vanskeligere når outputtet ikke er human-interpretable - f.eks. fordi det ikke finnes noe "riktig" output (slik som "The capitol of Norway is ...", som lar oss enkelt lage corrupted prompts for activation patching i Gpt-style modeller; "The capitol of Sweden is ..." - kanksje må vi bruke causal tracing i stedet, det gir ca. samme info, men uten behov for corrupted prompt). 

- Når modellen gir kontinuerlig output i stedet for diskret blir det vel mye vanskeligere å separere ut enkle circuits - om vi endrer inputtet litt kan andre circuits forskyve output i rommet, og fordi vi ikke har en forståelse av hva forskyvningen betyr kan vi ikke isolere som i "capitol of ..."-eksempelet. (Dessuten vil hele tekster kanksje inneholde mye mer interferens, men enkle strukturer som "er en vitenskapelig artikkel / har abstract/intro/metode..." bøør vel være mulig)

- Har vi dessuten en naturlig metrikk for å måle hvor mye outputtet har endret seg? Vi bør se på tapsfunksjonen for Jina AI for å finne noe rimelig (vi har altså ikke logits, med mindre vi trener og legger til noe alla et QA-lag ytterst)

- Kanksje er 'universality' i mindre grad sant for disse modellene da de ikke har en tydelig fasit, og da blir det vanskelig å generalisere fra den konkrete modellen (spesielt vil treningsdata definere om modellen fokuserer på strukturer i teksten eller tema i teksten, etc., når den legger tekster nærme hverandre... "some models might be topic-sensitive while others are structure-sensitive")

- Vil skrive kode som lar deg enkelt utføre: Ablation og activation patching på Jina AI. Vi må finne en god metrikk for encoder-only modeller, og vi må bruke modellens base-output som baseline. (Kanskje dette bare er å implementere causal scrubbing med "expected loss recovered, ie “what fraction of the expected loss on the subproblem we’re studying does our scrubbed circuit recover, compared to the original model with no edits”")

- Har vi funnet noen circuits fra før i Jina AI som vi kan teste på? Det er ikke klart at de fra GPT-stil transformers også dukker opp her (hverken previous token heads, pga. ALiBi, eller induction heads, pga. non-causal), men kanksje vi finner andre... MLP-circuits er ikke nødvendig vis annerledes.




#### Ane


Kan du si litt om hvordan du vil bruke notebooken? Eventuelt, prosjektbeskrivelse?

Skal vi gjøre *circuit detection* eller *interventions på pre-etablerte circuits*?

Hva bør input/output i notebooken være? 

Er det et spesifikt datasett vi bør analysere? En bestemt modell? 

Jina AI er encoder only / ikke kompatibel med tranformer_lens / naiv neuronpedia-implementering. Stemmer det at det er slike modeller du er interessert i? Har du noen ideer til metrikker (vi har jo ikke logits i slike modeller).

-Er vi opptatt av embeddingene til tekstene i det semantiske rommet (og bruker sammenhengen mellom embeddingene til å si noe om kretsene?)

-Attribution: man bruker en enkelt verdi (eg. classification)



## Glossary

**Transformers**
- Causal/autoregressive: Uses only previous tokens for prediction, achieved by masking the attention pattern.
- Logits: Output vector (length d_vocab) after unembedding but before softmax.
- LayerNorm: After transformer block we translate by mean value and normalize (to get variance 1, NB - this is non-linear) adding robustness param $\epsilon$, and then multiply by weights and add a bias, before passing on to next block.
- Attend to: We say that position $k$ attends to position $q$ if the corresp. attention pattern value is large.
- Residual stream: Main channel in transformer diagram, tracking the cumulative sum giving the updated embedding vector.

- Transformer config:
    * batch: input index in batch.
    * position: token position.
    * d_model: residual stream/embedding dimension
    * d_vocab: vocab size
    * n_ctx: context size
    * d_head: dimension of key/query space in attention head
    * d_mlp: mpl hidden layer size (4 x d_model)
    * n_heads: number of attention heads (d_model/d_head)
    * n_layers: number of layers
- Transformer params:
    * LayerNorm weights and biases
    * $W_E$: Embedding matrix (d_vocab, d_model)
    * $W_p$: Position embedding matrix, to give sequential information (n_ctx, d_model)
    * $W_U$: Unembedding matrix (d_model, d_vocab) [we may include bias to allow folding in LayerNorm]

    * W_{QK}: (product of two matrices - important for interpretability as the component-entries are entangled)
        * $W_Q$: Query matrix, for each head in each layer (n_heads, d_model, d_head)¨
        * $W_K$: Key matrix (n_heads, d_model, d_head)
    * W_{VO}:
        * $W_{V\downarrow}$ and $W_{O\uparrow}$: Value matrix (decomposed) (n_heads, d_model, d_head) and (n_heads, d_head, d_model)
    * Attention biases

    * MLP weights and biases (two layers / one hidden layer)

**Mech Interp**

*General hypotheses/concepts*
- Linear representation hypothesis (LH): LLMs choose to use (only) linear representations: Features correspond to directions (Nodes are basis vectors (because of non-linear activations picking out these as a privileged direction), and directions are linear combinations of them)
- Superposition: The model knows more concepts (features) than basis directions, hence each node encodes multiple concepts - we thus do not expect features to be represented by a single basis vector (challenge for interpretability). The number of almost orthogonal directions grows exponentially with dimension, so there is room for as many concepts as one would ever want in large models with minimal interference.
    - Polysemanticity: A single neuron looks for several (completely unrelated) things (eg for a visual model: cat faces, car fronts and cat legs - we know they are independent by feature visualisation). [Speculative: We want to 'unfold' polysemanticity with SAEs, decomposing into interpretable features in a higher dim space?]
- Universality (UH): Analagous features and circuits form across models and tasks (and even in biological brains). Mech Interp goal: 'Periodic table of features/circuits'. If untrue, Mech Interp will fail/should only focus on concrete models of high societal importance, but it seems mostly true.


*Methods/keywords*
- Activation patching: To identify important model activations, we define a performance metric and make a clean and a corrupted prompt, and pick a specific model activation. We run the model on the corrupted prompt, but intervene on the activation with the clean prompt values to see how much of original performance is restored by this activation.

- Attribution: the process of determining which parts of a model (tokens, features, neurons) are responsible for a given output. 
- Intervention: the act of changing the internal components of a model (like features or activations) to see how output changes.
- Hook point: Allows us to intervene/edit the corresp. activation. We apply a hook function which replaces the model activation with our desired intervention (the function output). We can extract an activation by returning nothing on the hook.
- Ablation/knockout: 'deleting' one activation (zero/mean/random ablation are different approaches)
- Toy model: Simple / very small model for easy interpretability.

- TransformerLens: Nanda's library for Mech Interp on GPT2-style LMs, specifically accessing/editing hook points in models ([demo](https://colab.research.google.com/github/neelnanda-io/TransformerLens/blob/main/demos/Main_Demo.ipynb#scrollTo=Eo1vbABrq9Ba)).

*Concepts*
- Features: Directions in semantic space / idealised nodes - a basic concept the model has grasped.
- Circuits: Connection of features by weights to grasp complex concepts (eg car = wheel + body + windows + doors etc.). Tightly linked features making a subgraph.
    - Ex: curve detectors, consisting of mulitple units to span all orientations, can be viewed as one idealised node; a feature, whcich later feeds into circuits which do eg circle detection.
- Feature visualisation: By using gradient descent with cost function maximizing the firing of a (known) feature/circuit, we can tweak the model input to maximally represent the feature (this uses LH, and gives a causal link).
- Feature implementation: A known circuit can be reimplemented by hand - if the behaviour remains, it is an isolated algorithm, like we want.

*Circuits* (sufficiently small subgraphs that one can make falsafiable predictions and reconstruct by hand)
- Motifs: Recurring abstract patterns in circuits such as equivariance, superposition, unioning:

    - Equivariant circuits: Exhibit a symmetry in the weights corresponding to a symmetry in the problem to be solved. For instance, a curve detector (orientation invariant) is composed of directed curve detectors, and these are essentially rotations of each other.

    - Unioning over cases: Often a concept is the union of several cases, eg the concept curve/dog is orientation invariant. The network might separately detect either (inhibiting each other on the way, XOR-properties) and then unionize at the end to get at the general concept.

    - Phenomenon superposition: A pure neuron is pushed forward to the output through superpositions to save on neurons (rather than keeping a separate 'trivial' stream encoding dog - dog - dog - ... until the output). Superposition is most useful when the concepts are mostly separate, so that the model can retrieve the correct concept without interference.

- Induction circuits: Circuit to detect/continue repeated subsequences. Composed of two consecutive heads (previous token head + induction head, which attends to the next token after previous instance of current token). Work for arbitrary length patterns. Not to hard to find, since they attend to tokens with same spacing.

-Advanced indexing: can use lists as indices in a tensor. 


**Jina AI** (BERT-style)
- non-causal encoder-style with contrastive training using a mean pooling layer ([arXiv](https://arxiv.org/pdf/2310.19923))
- NO positional embeddings, instead mirrored ALiBi (attention with linear bias)
- GLU for the FFNN (elementwise sigmoid - not sure of the details of the diffenrence, but it is wider (less interpretable?) than the std mlp archit.)
- post-layer normalization only
- In GPT-style, you get predicitons for all 

- Next-token prediction er et diskret problem, mens encoding er kontinuerlig - vi finner ikke analoger til induction circuits, der neste token i et repetert mønster er heldeterminert av en enkel induksjonskrets. 
- Blir PAD-tokens relevant i non-causal attention?
- Can we find few-shot learning heads?


- polysemanticity = standard basis is non-interpretable
- superposition = there is no interpretable basis (it is over-complete)
- Neurons and heads are distinct - in the computation graph view (feature graph), they may both be included.

## Articles/Resources

- Arena tutorials: https://arena3-chapter1-transformer-interp.streamlit.app/[1.2]_Intro_to_Mech_Interp

- Neel Nanda Quickstart Guide
- Neel Nanda Reading List

-Neel nanda glossary: https://www.neelnanda.io/mechanistic-interpretability/glossary


Demos/tutorials for neuronpedia: 


- Artikkel: https://transformer-circuits.pub/2025/attribution-graphs/biology.html

- Github: https://github.com/safety-research/circuit-tracer?tab=readme-ov-file

- Intervention demo: https://github.com/safety-research/circuit-tracer/blob/main/demos/intervention_demo.ipynb
- Attribution demo: https://github.com/safety-research/circuit-tracer/blob/main/demos/attribute_demo.ipynb
- Circuit tracing tutorial: https://github.com/safety-research/circuit-tracer/blob/main/demos/circuit_tracing_tutorial.ipynb


- Exploration demo: https://colab.research.google.com/github/neelnanda-io/TransformerLens/blob/main/demos/Exploratory_Analysis_Demo.ipynb

-TransformerLens activation and path patching (Neel Nanda): https://arena3-chapter1-transformer-interp.streamlit.app/[1.4.1]_Indirect_Object_Identification

-Logit lens: https://www.alignmentforum.org/posts/AcKRB8wDpdaN6v6ru/interpreting-gpt-the-logit-lens


-Activation patching: https://arxiv.org/pdf/2404.15255

-Dataset: https://huggingface.co/datasets/cindermond/bento/blob/e45bde13c992895825817b84a4c47dd4c53b7152/agieval_reduced/aqua-rat/test.jsonl
https://gitlab.info.uqam.ca/guite-vinet.julien/inf7470-project2/-/blob/master/src/data/AQuA-master/test.tok.json
https://github.com/google-deepmind/AQuA/blob/master/test.tok.json


-Source code, transformers.modeling_bert: 
https://huggingface.co/transformers/v2.1.1/_modules/transformers/modeling_bert.html?utm_source=chatgpt.com

-Path patching: https://github.com/callummcdougall/SERIMATS_app/blob/main/path_patching.py
https://github.com/callummcdougall/path_patching

https://colab.research.google.com/drive/1yV6czsp9EWWGORRIeGrdQDdJJ8b7EI4n#scrollTo=Uejux10Bl6mW
https://colab.research.google.com/drive/15CJ1WAf8AWm6emI3t2nVfnO85-hxwyJU#scrollTo=Zx2LqFhJXxhw
Interpretability in the wild: https://arxiv.org/pdf/2211.00593

-Semantic search: https://www.sbert.net/examples/sentence_transformer/applications/semantic-search/README.html





-Mechanistic interpretability for AI safety: https://leonardbereska.github.io/blog/2024/mechinterpreview/

-Attribution pathcing: https://www.neelnanda.io/mechanistic-interpretability/attribution-patching
- Demo: https://colab.research.google.com/github/neelnanda-io/TransformerLens/blob/main/demos/Attribution_Patching_Demo.ipynb

-Causal scrubbing: https://www.alignmentforum.org/posts/JvZhhzycHu2Yd57RN/causal-scrubbing-a-method-for-rigorously-testing

-ACDC: https://arxiv.org/pdf/2304.14997

TODO
- pysvelte interactive visualisations / attention patterns
- Neel Nanda reading list (2024, potentially outdated): https://www.alignmentforum.org/posts/NfFST5Mio7BCAQHPA/an-extremely-opinionated-annotated-list-of-my-favourite-1
    - Sparse autoencoders
    - Circuits
  - Arena tutorials?
  - Neel Nanda videos
- Weekend: Start coding/implementing

- Attribution: https://arxiv.org/pdf/1703.01365

